In [450]:
import pandas as pd
import math

In [451]:
data = pd.read_csv('cars.csv')
data.head()

,buying,maint,doors,persons,lug_boot,safety,clazz
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [452]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [453]:
classes = list(data['clazz'].unique())
features = list(data.keys())[:6]

In [454]:
def build(data):
    
    table = dict()

    cnts = data['clazz'].value_counts()
    table['class_names'] = list(cnts.index)
    table['class_counts'] = list(cnts.values)

    for feature in features:
        table[feature] = dict()

        counts = data.groupby('clazz')[feature].value_counts()
        data_counts = counts.unstack('clazz')

        # correction factor

        # if data_counts.isna().any(axis=None):
        #     data_counts.fillna(value=0, inplace=True)
        #     data_counts += 1
        
        data_probs = data_counts / data_counts.sum()

        for value in data_probs.index:
            probabilities = data_probs.loc[value]
            probs = dict()
            for i in classes:
                probs[i] = 0
            for i in classes:
                if not math.isnan(probabilities[i]):
                    probs[i] = probabilities[i]
            table[feature][value] = probs
            
    return table  

In [455]:
table = build(train)

In [456]:
def naive_bayes_predictor(row, table):
    class_probs = dict()
    
    for i in range(len(classes)):
        class_probs[table['class_names'][i]] = table['class_counts'][i]

    for feature in features:
        value = row[feature]
        probs = table[feature][value]
        for i in classes:
            class_probs[i] *= probs[i]
    
    max_value = 0
    answer = ""
    for i in class_probs.keys():
        if max_value < class_probs[i]:
            max_value = class_probs[i]
            answer = i
    
    return answer

In [457]:
correct = 0

for i in range(len(test)):
    current_row = test.iloc[i]
    prediction = naive_bayes_predictor(current_row, table)
    if current_row['clazz'] == prediction:
        correct += 1

print("Accuracy is ", correct / len(test))

Accuracy is  0.8670520231213873
